# Neural Network Model


In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

# Import and read the csv.
# # b1 view
# df = pd.read_csv('ca_b2_df.csv', index_col=[0])
# df.head()

# b2 view
df = pd.read_csv('ML Data/ca_b2_df.csv', index_col=[0])
df.head()

# # b3 view
# df = pd.read_csv('ca_b2_df.csv', index_col=[0])
# df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ML Data/ca_b2_df.csv'

In [2]:
df['leads'].value_counts()

1    9568
0    6464
Name: leads, dtype: int64

# Split into train and test features

In [3]:
# Split our preprocessed data into our features and target arrays
y = df['leads'].values
X = df.drop(['leads'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\dia78039\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, Evaluate our Model 1 - NN

In [5]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# using multi-layer perceptron (two layers)
numInputFeatures = len(X_train[0])

# I have X amount of columns
# layer1 = input layer, typically equals number of input variables in data
layer1 = 300
# layer 2 = hidden layer, typically 2/3 of input layer
layer2 = 150
# layer 3 = hidden layer
layer3= 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1, 
                          input_dim=numInputFeatures, 
                          activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, 
                             activation="relu"))

# adding a third layer to increase accuracy 
nn.add(tf.keras.layers.Dense(units=layer3, 
                             activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               1800      
                                                                 
 dense_1 (Dense)             (None, 150)               45150     
                                                                 
 dense_2 (Dense)             (None, 20)                3020      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 49,991
Trainable params: 49,991
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    # checkpoint directory and file structure defined above
    filepath=checkpoint_path,
    # notified when checkpoint is being saved to the directory
    verbose=1,
    # checkpoint files take small space
    save_weights_only=True,
    # checkpoints saved every epoch
    save_freq='epoch')

In [7]:
# Train the model
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
348/376 [==========================>...] - ETA: 0s - loss: 0.4526 - accuracy: 0.7934
Epoch 1: saving model to checkpoints\weights.01.hdf5
376/376 [==============================] - 1s 920us/step - loss: 0.4517 - accuracy: 0.7942
Epoch 2/100
324/376 [========================>.....] - ETA: 0s - loss: 0.4304 - accuracy: 0.8010
Epoch 2: saving model to checkpoints\weights.02.hdf5
376/376 [==============================] - 0s 965us/step - loss: 0.4309 - accuracy: 0.7996
Epoch 3/100
324/376 [========================>.....] - ETA: 0s - loss: 0.4285 - accuracy: 0.8013
Epoch 3: saving model to checkpoints\weights.03.hdf5
376/376 [==============================] - 0s 1ms/step - loss: 0.4277 - accuracy: 0.8001
Epoch 4/100
347/376 [==========================>...] - ETA: 0s - loss: 0.4216 - accuracy: 0.8034
Epoch 4: saving model to checkpoints\weights.04.hdf5
376/376 [==============================] - 0s 1ms/step - loss: 0.4235 - accuracy: 0.8024
Epoch 5/100
343/376 [===================

Epoch 35/100
339/376 [==========================>...] - ETA: 0s - loss: 0.4125 - accuracy: 0.8057
Epoch 35: saving model to checkpoints\weights.35.hdf5
376/376 [==============================] - 0s 935us/step - loss: 0.4134 - accuracy: 0.8055
Epoch 36/100
337/376 [=========================>....] - ETA: 0s - loss: 0.4143 - accuracy: 0.8050
Epoch 36: saving model to checkpoints\weights.36.hdf5
376/376 [==============================] - 0s 936us/step - loss: 0.4123 - accuracy: 0.8066
Epoch 37/100
335/376 [=========================>....] - ETA: 0s - loss: 0.4117 - accuracy: 0.8072
Epoch 37: saving model to checkpoints\weights.37.hdf5
376/376 [==============================] - 0s 940us/step - loss: 0.4122 - accuracy: 0.8061
Epoch 38/100
333/376 [=========================>....] - ETA: 0s - loss: 0.4132 - accuracy: 0.8038
Epoch 38: saving model to checkpoints\weights.38.hdf5
376/376 [==============================] - 0s 945us/step - loss: 0.4120 - accuracy: 0.8041
Epoch 39/100
335/376 [======

Epoch 69/100
342/376 [==========================>...] - ETA: 0s - loss: 0.4076 - accuracy: 0.8068
Epoch 69: saving model to checkpoints\weights.69.hdf5
376/376 [==============================] - 0s 925us/step - loss: 0.4054 - accuracy: 0.8077
Epoch 70/100
342/376 [==========================>...] - ETA: 0s - loss: 0.4055 - accuracy: 0.8091
Epoch 70: saving model to checkpoints\weights.70.hdf5
376/376 [==============================] - 0s 919us/step - loss: 0.4057 - accuracy: 0.8083
Epoch 71/100
342/376 [==========================>...] - ETA: 0s - loss: 0.4044 - accuracy: 0.8078
Epoch 71: saving model to checkpoints\weights.71.hdf5
376/376 [==============================] - 0s 923us/step - loss: 0.4050 - accuracy: 0.8084
Epoch 72/100
323/376 [========================>.....] - ETA: 0s - loss: 0.4072 - accuracy: 0.8065
Epoch 72: saving model to checkpoints\weights.72.hdf5
376/376 [==============================] - 0s 965us/step - loss: 0.4046 - accuracy: 0.8084
Epoch 73/100
346/376 [======

In [8]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

126/126 - 0s - loss: 0.4323 - accuracy: 0.7934 - 169ms/epoch - 1ms/step
Loss: 0.43231311440467834, Accuracy: 0.7934131622314453


In [10]:
# # Export the model to HDF5 file
# nn.save("AlphabetSoupCharity_optimization.h5")